# Enfocado a seleccionar el mejor valor de la variable de "Tipo_de_Fuente".

In [82]:
import pandas as pd

In [83]:
contaminantes = pd.read_csv("..\data\Contaminantes\calidad_aire_2025.csv")
contaminantes.head()

,Entidad_federativa,Municipio,Tipo_de_Fuente,SO_2,CO,NOx,COV,PM_010,PM_2_5,NH_3,Entidad
0,Aguascalientes,Aguascalientes,Fuentes fijas,546.316,94.308,209.404,2219.396,246.873,182.143,2.929,Aguascalientes
1,Aguascalientes,Aguascalientes,Fuentes de área,15.877,2396.298,333.306,12438.827,1567.037,483.104,13661.221,Aguascalientes
2,Aguascalientes,Aguascalientes,Fuentes móviles carreteros,308.001,48527.124,10486.580,5148.961,721.389,659.929,83.887,Aguascalientes
3,Aguascalientes,Aguascalientes,Fuentes móviles que no circulan por carretera,36.981,548.796,1042.305,96.899,66.318,63.940,0.226,Aguascalientes
4,Aguascalientes,Aguascalientes,Fuentes naturales,NaN,NaN,440.268,1663.833,NaN,NaN,NaN,Aguascalientes


In [84]:
print(contaminantes.shape)
print()

valores = contaminantes["Tipo_de_Fuente"].unique()
print(valores)

(10521, 11)

['Fuentes fijas' 'Fuentes de área' 'Fuentes móviles carreteros'
 'Fuentes móviles que no circulan por carretera' 'Fuentes naturales']


## Aproximación: Descartar valores en función de la respectiva cantidad de valores nulos y cantidad de observaciones.

In [85]:
for i in valores:
    print("==========")
    print(i)
    print("==========")
    aux = contaminantes[contaminantes["Tipo_de_Fuente"] == i]
    print(aux.isna().sum())
    print()
    print(f"Observaciones: {len(aux)}")
    print(f"Cantidad NAN's: {aux.isna().sum().sum()}")
    print()

Fuentes fijas
Entidad_federativa    0
Municipio             0
Tipo_de_Fuente        0
SO_2                  0
CO                    0
NOx                   0
COV                   0
PM_010                0
PM_2_5                0
NH_3                  0
Entidad               0
dtype: int64

Observaciones: 700
Cantidad NAN's: 0

Fuentes de área
Entidad_federativa    0
Municipio             0
Tipo_de_Fuente        0
SO_2                  0
CO                    0
NOx                   0
COV                   0
PM_010                0
PM_2_5                0
NH_3                  0
Entidad               0
dtype: int64

Observaciones: 2459
Cantidad NAN's: 0

Fuentes móviles carreteros
Entidad_federativa    0
Municipio             0
Tipo_de_Fuente        0
SO_2                  0
CO                    0
NOx                   0
COV                   0
PM_010                0
PM_2_5                0
NH_3                  0
Entidad               0
dtype: int64

Observaciones: 2450
Cantidad NAN

In [86]:
contaminantes.isna().sum().sum()

12265

Se observa que todos los valores nulos del DataFrame "contaminantes" pertenecen a las observaciones con "Tipo_de_Fuente" igual a "Fuentes naturales", por lo que es recomendable descartar este valor. Por la poca cantidad de observaciones del valor "Fuentes fijas" es recomendable descartarlo.

In [87]:
valores = [v for v in valores if v not in ["Fuentes naturales", "Fuentes fijas"]]
print(valores)

['Fuentes de área', 'Fuentes móviles carreteros', 'Fuentes móviles que no circulan por carretera']


## Aproximación: Priorizando la varianza, seleccionaremos el valor de "Tipo_de_Fuente" que obtenga mayor numero de veces en las que la varianza de cada gas sea la maxima comparada con la varianza del mismo gas pero de diferente "Tipo_de_Fuente".

In [88]:
columnas = contaminantes.drop(columns=["Entidad_federativa", "Municipio", "Tipo_de_Fuente", "Entidad"]).columns

best = {}
for col in columnas:
    best[f"val_{col}"] = 0
    best[f"fuente_{col}"] = "Nada"

for fuente in valores:
    print("==========")
    print(fuente)
    print("==========")
    aux = contaminantes[contaminantes["Tipo_de_Fuente"] == fuente]
    print(aux.drop(columns=["Entidad_federativa", "Municipio", "Tipo_de_Fuente", "Entidad"]).var().round(2))
    for col in columnas:
        var_actual = aux[col].var()
        if var_actual > best[f"val_{col}"]:
            best[f"val_{col}"] = var_actual
            best[f"fuente_{col}"] = fuente   
    print()

Fuentes de área


SO_2          509.17
CO        6275773.27
NOx         11635.42
COV       4359995.90
PM_010     231067.41
PM_2_5      83574.89
NH_3       865681.49
dtype: float64

Fuentes móviles carreteros
SO_2          1583.01
CO        57026581.08
NOx        6028400.90
COV         631033.39
PM_010       14023.26
PM_2_5       11093.86
NH_3           229.09
dtype: float64

Fuentes móviles que no circulan por carretera
SO_2         84.30
CO        18038.32
NOx       28269.89
COV        1196.11
PM_010      112.10
PM_2_5      105.05
NH_3          0.00
dtype: float64



In [89]:
print("==========")
print("Varianzas más grandes por columna")
print("==========")
for col in columnas:
    print(f"{col}: {best[f'val_{col}']:.2f}")
    print(f"{best[f'fuente_{col}']}")
    print()

Varianzas más grandes por columna
SO_2: 1583.01
Fuentes móviles carreteros

CO: 57026581.08
Fuentes móviles carreteros

NOx: 6028400.90
Fuentes móviles carreteros

COV: 4359995.90
Fuentes de área

PM_010: 231067.41
Fuentes de área

PM_2_5: 83574.89
Fuentes de área

NH_3: 865681.49
Fuentes de área



Por lo tanto el valor seleccionado será Tipo_de_Fuente = "Fuentes de área".